In [ ]:
from netgen import gui
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
import numpy as np

In [ ]:
def solve_problem(i, j, theta = 0.5):
    geo = SplineGeometry()
    geo.AddRectangle( (0, 0), (1, 1),
                    bcs = ("bottom", "right", "top", "left"))
    mesh = Mesh( geo.GenerateMesh(maxh=0.5**i))
    Draw(mesh)
    fes = H1(mesh, order=4, dirichlet="bottom|right|left|top")

    u,v = fes.TnT()

    time = 0.0
    dt = 0.5**j

    s = BilinearForm(fes, symmetric=False)
    s += grad(u)*grad(v)*dx
    s.Assemble()

    m = BilinearForm(fes, symmetric=False)
    m += u*v*dx
    m.Assemble()

    mstar = m.mat.CreateMatrix()
    mstar.AsVector().data = m.mat.AsVector() + (theta)* dt * s.mat.AsVector()
    invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
    
    t = Parameter(0.0)
    fnow = LinearForm(fes)
    fnow += sin(2*pi*x)*sin(2*pi*y)*(8*pi**2*cos(4*pi*t) - 4*pi*sin(4*pi*t))*v*dx
    fnow.Assemble()
    fnext = LinearForm(fes)
    fnext += sin(2*pi*x)*sin(2*pi*y)*(8*pi**2*cos(4*pi*(t+dt)) - 4*pi*sin(4*pi*(t+dt)))*v*dx
    fnext.Assemble()

    #u = lambda t: sin(2*pi*x)*sin(2*pi*y)*exp(-8*pi**2*t)
    u = lambda t: sin(2*pi*x)*sin(2*pi*y)*cos(4*pi*t)

    gfu = GridFunction(fes)
    gfu.Set(u(0.))
    Draw(gfu,mesh,"u")

    res = gfu.vec.CreateVector()
    tstep = 1 # time that we want to step over within one block-run
    
    l2errormax = 0
    t_intermediate=0 # time counter within one block-run
    while t_intermediate < tstep - 0.5 * dt:
        t.Set(t_intermediate)
        fnow.Assemble()
        fnext.Assemble()
        res.data = (1-theta)*dt * fnow.vec + (theta)*dt*fnext.vec - dt * s.mat * gfu.vec
        gfu.vec.data += invmstar * res
        t_intermediate += dt
        
        l2error = sqrt( Integrate((gfu-u(t_intermediate))**2, mesh))
        if l2error > l2errormax:
            l2errormax = l2error
        #print("t: ",time+t_intermediate,"\t", "error: ", l2error)
        #Redraw(blocking=True)
    return l2errormax

In [ ]:
solve_problem(5,5)

In [ ]:
N = 1
M = 5
errors = np.zeros((N,M))
for i in range(N):
    for j in range(M):
        errors[i,j] = solve_problem(i+4,j+2, 0.500001)
print(errors)

In [ ]:
l2errors_dt_ref = errors[-1,:]
eocs_dt = []
for j in range(1,len(l2errors_dt_ref)):
    eocs_dt.append(log(l2errors_dt_ref[j-1]/l2errors_dt_ref[j])/log(2))
print(l2errors_dt_ref)
print(eocs_dt)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

plt.yscale("log")
ax.plot(l2errors_dt_ref, label="data")

ax.plot([0.5**(i+4) for i in range(len(l2errors_dt_ref))], label="First order")
ax.plot([0.5**(2*i+5) for i in range(len(l2errors_dt_ref))], label="Second order")

plt.legend(loc='lower left')
plt.show()